In [58]:
import pandas as pd
from urllib.parse import unquote
import unicodedata

# ========================
# FUNÇÕES DE LIMPEZA (BLOCO 1 e 3)
# ========================

def limpar_texto(texto):
    # ... (Sua função limpar_texto) ...
    if isinstance(texto, str):
        texto = texto.strip().lower()
        texto = ''.join(
            c for c in unicodedata.normalize('NFD', texto)
            if unicodedata.category(c) != 'Mn'
        )
    return texto

def limpar_categoria(df, coluna="Categoria"):
    df[coluna] = df[coluna].astype(str)
    extraido = df[coluna].str.extract(r"q=([^&]+)", expand=False)
    df["categoria_limpa"] = extraido.fillna(df[coluna])
    df["categoria_limpa"] = df["categoria_limpa"].apply(lambda x: unquote(x))
    df["categoria_limpa"] = df["categoria_limpa"].str.replace("+", " ", regex=False)
    df["categoria_limpa"] = df["categoria_limpa"].apply(limpar_texto)
    return df

def normalize_colname(s):
    s = str(s)
    s = unicodedata.normalize("NFKD", s).encode("ASCII", "ignore").decode()
    s = s.strip().lower().replace(" ", "_").replace("-", "_")
    return s

# ========================
# 1. LEITURA, PROCESSAMENTO E CONCATENAÇÃO
# ========================
arquivo = "mercados.xlsx"
abas = pd.read_excel(arquivo, sheet_name=None)
dfs_processados = []

for nome_aba, df in abas.items():
    print("Processando:", nome_aba)

    # Limpeza de Categoria
    df = limpar_categoria(df, coluna="Categoria")
    df["Categoria"] = df["categoria_limpa"]
    df = df.drop(columns=["categoria_limpa"])

    # Conversão de Data
    df["Data da Coleta"] = pd.to_datetime(
        df["Data da Coleta"], dayfirst=True, errors="coerce"
    )

    # Adicionar nome do supermercado
    df["supermercado"] = nome_aba
    dfs_processados.append(df)

# Unir tudo no DataFrame FINAL
df_final = pd.concat(dfs_processados, ignore_index=True)
print("Processamento e Concatenação concluídos.")

# ========================
# 2. PADRONIZAÇÃO DE COLUNAS (BASEADO NO BLOCO 3, SEM A RE-LEITURA)
# ========================

# Usar o DataFrame FINAL do passo 1
df = df_final

# Normalizar nomes de colunas
df.columns = [normalize_colname(c) for c in df.columns]

# Checagem de colunas: agora deve passar, pois 'supermercado' existe
required = ["produto", "preco", "categoria", "data_da_coleta", "supermercado"]
missing = [c for c in required if c not in df.columns]
if missing:
    # Este erro não deve mais acontecer!
    raise ValueError(f"Faltam colunas esperadas: {missing}. Colunas atuais: {list(df.columns)}")

# --- Conversão de data (apenas para garantir) ---
df["data_da_coleta"] = pd.to_datetime(df["data_da_coleta"], errors="coerce")

# --- Limpeza robusta da coluna de preço ---
# Seu código de limpeza de preço...
df["preco"] = df["preco"].astype(str)
df["preco_num"] = (
    df["preco"]
    .str.replace("R\\$", "", regex=True)
    .str.replace("\\s+", "", regex=True)
    .str.extract(r"(\d+(?:[\.,]\d+)?)")[0]
    .str.replace(",", ".", regex=False)
)
df["preco"] = pd.to_numeric(df["preco_num"], errors="coerce")
df = df.drop(columns=["preco_num"])
df = df.dropna(subset=["preco"]) # Remover linhas sem preço válido

# --- Filtrar período desejado ---
start = "2025-09-10"
end = "2025-10-10"
df = df[(df["data_da_coleta"] >= start) & (df["data_da_coleta"] <= end)]

# --- Agrupar por supermercado e categoria e calcular preço médio ---
categoria_resumo = (
    df.groupby(["supermercado", "categoria"])["preco"]
      .mean()
      .reset_index()
      .rename(columns={"preco": "preco_medio"})
)

print("\nProcessamento FINALizado com sucesso!")
print("\nPrimeiras linhas do Resumo:")
print(categoria_resumo.head())

Processando: Amigão
Processando: São_Judas_Tadeu
Processando: Pão_de_açúcar
Processamento e Concatenação concluídos.

Processamento FINALizado com sucesso!

Primeiras linhas do Resumo:
  supermercado             categoria  preco_medio
0       Amigão          acougue para    18.053392
1       Amigão            acucar 1kg    10.557617
2       Amigão            acucar 5kg    12.500608
3       Amigão  agua mineral com gas     7.164293
4       Amigão             arroz 1kg    14.873333


In [59]:
import pandas as pd
import unicodedata
from urllib.parse import unquote

# ========================
# RE-DEFINIÇÃO DAS SUAS FUNÇÕES (Para garantir que estão no escopo)
# ========================
def limpar_texto(texto):
    if isinstance(texto, str):
        texto = texto.strip().lower()
        texto = ''.join(
            c for c in unicodedata.normalize('NFD', texto)
            if unicodedata.category(c) != 'Mn'
        )
    return texto

def limpar_categoria(df, coluna="categoria"): # Coluna mudada para 'categoria' minúscula
    df[coluna] = df[coluna].astype(str)

    # Extrair valor após q= (para URLs que funcionam)
    extraido = df[coluna].str.extract(r"q=([^&]+)", expand=False)

    # Decodificar URL e remover '+'
    df["categoria_limpa"] = extraido.fillna(df[coluna])
    df["categoria_limpa"] = df["categoria_limpa"].apply(lambda x: unquote(x))
    df["categoria_limpa"] = df["categoria_limpa"].str.replace("+", " ", regex=False)

    # Limpeza final (normalização de acentos, minúsculas, etc.)
    df["categoria_limpa"] = df["categoria_limpa"].apply(limpar_texto)

    return df

# ========================
# SEU DICIONÁRIO COMPLETO
# ========================
map_categoria = {
    "acougue para": "açougue",
    "feijao": "feijão",
    "frango": "açougue",
    "iogurte": "iogurte",
    "leite": "leite",
    "macarrao": "massas",
    "macarrao espaguete": "massas",
    "oleo de soja": "oleo de soja",
    "oleo soja": "oleo de soja",
    "ovos": "ovos",
    "ovos de galinha": "ovos",
    "pao de forma": "padaria",
    "pao forma": "padaria",
    "queijo": "queijo",
    "refrigerante 2 litros": "bebidas não alcoolicas",
    "refrigerante 2l": "bebidas não alcoolicas",
    "sal 1kg": "sal",
    "salada organica": "hortifruti",
    "sopa": "sopa",
    "sorvete": "sorvete",
    "vinho tinto": "bebidas alcoolicas",

    "https://compreonline.supersaojudas.com.br/loja6/frutas-18/": "hortifruti",
    "https://compreonline.supersaojudas.com.br/loja6/legumes-62/": "hortifruti",
    "https://compreonline.supersaojudas.com.br/loja6/verduras-40/": "hortifruti",
    "https://www.amigao.com/hortifruti/?category-1=alimentos&category-2=hortifruti&category-3=frutas&facets=category-1,category-2,category-3&sort=score_desc&page=0": "hortifruti",
    "https://www.amigao.com/hortifruti/?category-1=alimentos&category-2=hortifruti&category-3=legumes-e-vegetais&facets=category-1,category-2,category-3&sort=score_desc&page=0": "hortifruti",
    "https://www.amigao.com/hortifruti/?category-1=alimentos&category-2=hortifruti&category-3=verduras&facets=category-1,category-2,category-3&sort=score_desc&page=0": "hortifruti",
    "https://www.paodeacucar.com/busca?terms=acucar  1kg": "açucar 1kg",
    "https://www.paodeacucar.com/busca?terms=acucar  5kg": "açucar 5kg",
    "https://www.paodeacucar.com/busca?terms=agua mineral com gas": "bebidas não alcoolicas",
    "https://www.paodeacucar.com/busca?terms=arroz 1kg": "arroz 1kg",
    "https://www.paodeacucar.com/busca?terms=arroz 5kg": "arroz 5kg",
    "https://www.paodeacucar.com/busca?terms=cafe 500g": "cafe",
    "https://www.paodeacucar.com/busca?terms=cerveja lata 350ml": "bebidas alcoolicas",
    "https://www.paodeacucar.com/busca?terms=cha": "bebidas não alcoolicas",
    "https://www.paodeacucar.com/busca?terms=chocolate &p=3": "chocolate",
    "https://www.paodeacucar.com/busca?terms=feijao": "feijão",
    "https://www.paodeacucar.com/busca?terms=iogurte": "iogurte",
    "https://www.paodeacucar.com/busca?terms=leite 1l": "leite",
    "https://www.paodeacucar.com/busca?terms=macarrao&p=2": "massas",
    "https://www.paodeacucar.com/busca?terms=oleo de soja": "oleo de soja",
    "https://www.paodeacucar.com/busca?terms=ovos": "ovos",
    "https://www.paodeacucar.com/busca?terms=pao de forma": "padaria",
    "https://www.paodeacucar.com/busca?terms=queijo&p=3": "queijo",
    "https://www.paodeacucar.com/busca?terms=sal": "sal",
    "https://www.paodeacucar.com/busca?terms=salada &p=2": "hortifruti",
    "https://www.paodeacucar.com/busca?terms=sopa": "sopa",
    "https://www.paodeacucar.com/busca?terms=sorvete&p=3": "sorvete",
    "https://www.paodeacucar.com/categoria/alimentos/acougue/carnes/carne-aves?s=relevance&p=2": "açougue",
    "https://www.paodeacucar.com/categoria/alimentos/acougue/carnes/carne-bovina?s=relevance&p=1": "hortifruti",
    "https://www.paodeacucar.com/categoria/alimentos/hortifruti/frutas?s=relevance&p=1": "hortifruti",
    "https://www.paodeacucar.com/categoria/alimentos/hortifruti/legumes-e-verduras/hortalicas?s=relevance&p=3": "hortifruti",
    "https://www.paodeacucar.com/categoria/alimentos/hortifruti/legumes-e-verduras/legumes?s=relevance&p=1": "hortifruti",
    "https://www.paodeacucar.com/categoria/bebidas/nao-alcoolicas/refrigerantes?s=relevance&p=3": "bebidas não alcoolicas",
    "https://www.paodeacucar.com/especial/tinto-app?p=4": "bebidas alcoolicas",
    "https://www.paodeacucar.com/busca?terms=acucar_1kg" : "açucar 1kg",
    "https://www.paodeacucar.com/busca?terms=acucar_5kg" : "açucar 5kg"

}

# --- ASSUME QUE O DATAFRAME COMBINADO É CHAMADO 'df' ---

# --- 1. Limpeza Forçada (Para resolver os problemas de URLs incompletas) ---
df['categoria'] = df['categoria'].astype(str)

# Limpeza específica para as strings problemáticas (remove números no final e URLs que falharam na extração 'q=')
df['categoria'] = df['categoria'].str.replace(r'\d+https.*', '', regex=True)
df['categoria'] = df['categoria'].str.strip()

# 2. Reaplica a função limpar_categoria (essencial)
# Ela extrai o texto do 'q=' e normaliza (remove acentos, minúsculas)
df = limpar_categoria(df, coluna="categoria")
df["categoria"] = df["categoria_limpa"]
df = df.drop(columns=["categoria_limpa"])

# --- 3. Mapeamento e Agrupamento ---

# Mapeamento
df["categoria_agrupada"] = df["categoria"].map(map_categoria)

# Substitui o original (mantendo o nome original se o mapeamento falhar/for NaN)
df["categoria"] = df["categoria_agrupada"].fillna(df["categoria"])

# Remover coluna temporária
df = df.drop(columns=["categoria_agrupada"])

print("✅ Limpeza e Mapeamento concluídos. Verificando resultados...")

# --- Verificação Final ---
print("\n### 📊 Contagem de Linhas por Categoria Agrupada (TOP 20):")
print(df["categoria"].value_counts().head(20))

✅ Limpeza e Mapeamento concluídos. Verificando resultados...

### 📊 Contagem de Linhas por Categoria Agrupada (TOP 20):
categoria
hortifruti                11239
bebidas não alcoolicas     9631
queijo                     4328
sal                        4157
bebidas alcoolicas         3929
massas                     3865
açougue                    2958
padaria                    2778
leite                      2721
arroz 5kg                  2713
arroz 1kg                  2663
ovos                       2554
feijão                     2401
iogurte                    2192
cafe                       1945
sorvete                    1738
cafe 500g                  1199
cha                        1160
chocolate                   911
sopa                        830
Name: count, dtype: int64


In [52]:
# Exibe as primeiras linhas da tabela combinada
print(df.head())

                                             produto  preco categoria  \
0              Leite UHT Integral Tirol com Tampa 1L   4.99     leite   
1              Leite Pasteurizado Integral Batavo 1L   4.99     leite   
2                          Leite Desnatado Molico 1L   6.98     leite   
3           Leite Integral UHT Zero Lactose Tirol 1L   5.39     leite   
4  Leite Uht Semidesnatado Zero Lactose Líder Cai...   5.39     leite   

       data_da_coleta supermercado  
0 2025-09-10 15:28:40       Amigão  
1 2025-09-10 15:28:40       Amigão  
2 2025-09-10 15:28:40       Amigão  
3 2025-09-10 15:28:40       Amigão  
4 2025-09-10 15:28:40       Amigão  


In [53]:
categorias = sorted(df["categoria"].unique())

template = {cat: "" for cat in categorias}

template

{'acougue para': '',
 'acucar 1kg': '',
 'acucar 5kg': '',
 'agua mineral c gas': '',
 'agua mineral com gas': '',
 'arroz 1kg': '',
 'arroz 5kg': '',
 'cafe 500g': '',
 'carne': '',
 'carne bovina': '',
 'cerveja': '',
 'cerveja lata 350ml': '',
 'cha': '',
 'chocolate 80g': '',
 'chocolate barra': '',
 'feijao': '',
 'frango': '',
 'https://compreonline.supersaojudas.com.br/loja6/frutas-18/': '',
 'https://compreonline.supersaojudas.com.br/loja6/legumes-62/': '',
 'https://compreonline.supersaojudas.com.br/loja6/verduras-40/': '',
 'https://www.amigao.com/hortifruti/?category-1=alimentos&category-2=hortifruti&category-3=frutas&facets=category-1,category-2,category-3&sort=score_desc&page=0': '',
 'https://www.amigao.com/hortifruti/?category-1=alimentos&category-2=hortifruti&category-3=legumes-e-vegetais&facets=category-1,category-2,category-3&sort=score_desc&page=0': '',
 'https://www.amigao.com/hortifruti/?category-1=alimentos&category-2=hortifruti&category-3=verduras&facets=category

In [54]:
map_categoria = {
    "acougue para": "açougue",
    "feijao": "feijão",
    "frango": "açougue",
    "iogurte": "iogurte",
    "leite": "leite",
    "macarrao": "massas",
    "macarrao espaguete": "massas",
    "oleo de soja": "oleo de soja",
    "oleo soja": "oleo de soja",
    "ovos": "ovos",
    "ovos de galinha": "ovos",
    "pao de forma": "padaria",
    "pao forma": "padaria",
    "queijo": "queijo",
    "refrigerante 2 litros": "bebidas não alcoolicas",
    "refrigerante 2l": "bebidas não alcoolicas",
    "sal 1kg": "sal",
    "salada organica": "hortifruti",
    "sopa": "sopa",
    "sorvete": "sorvete",
    "vinho tinto": "bebidas alcoolicas",

    "https://compreonline.supersaojudas.com.br/loja6/frutas-18/": "hortifruti",
    "https://compreonline.supersaojudas.com.br/loja6/legumes-62/": "hortifruti",
    "https://compreonline.supersaojudas.com.br/loja6/verduras-40/": "hortifruti",
    "https://www.amigao.com/hortifruti/?category-1=alimentos&category-2=hortifruti&category-3=frutas&facets=category-1,category-2,category-3&sort=score_desc&page=0": "hortifruti",
    "https://www.amigao.com/hortifruti/?category-1=alimentos&category-2=hortifruti&category-3=legumes-e-vegetais&facets=category-1,category-2,category-3&sort=score_desc&page=0": "hortifruti",
    "https://www.amigao.com/hortifruti/?category-1=alimentos&category-2=hortifruti&category-3=verduras&facets=category-1,category-2,category-3&sort=score_desc&page=0": "hortifruti",
    "https://www.paodeacucar.com/busca?terms=acucar  1kg": "açucar 1kg",
    "https://www.paodeacucar.com/busca?terms=acucar  5kg": "açucar 5kg",
    "https://www.paodeacucar.com/busca?terms=agua mineral com gas": "bebidas não alcoolicas",
    "https://www.paodeacucar.com/busca?terms=arroz 1kg": "arroz 1kg",
    "https://www.paodeacucar.com/busca?terms=arroz 5kg": "arroz 5kg",
    "https://www.paodeacucar.com/busca?terms=cafe 500g": "cafe",
    "https://www.paodeacucar.com/busca?terms=cerveja lata 350ml": "bebidas alcoolicas",
    "https://www.paodeacucar.com/busca?terms=cha": "bebidas não alcoolicas",
    "https://www.paodeacucar.com/busca?terms=chocolate &p=3": "chocolate",
    "https://www.paodeacucar.com/busca?terms=feijao": "feijão",
    "https://www.paodeacucar.com/busca?terms=iogurte": "iogurte",
    "https://www.paodeacucar.com/busca?terms=leite 1l": "leite",
    "https://www.paodeacucar.com/busca?terms=macarrao&p=2": "massas",
    "https://www.paodeacucar.com/busca?terms=oleo de soja": "oleo de soja",
    "https://www.paodeacucar.com/busca?terms=ovos": "ovos",
    "https://www.paodeacucar.com/busca?terms=pao de forma": "padaria",
    "https://www.paodeacucar.com/busca?terms=queijo&p=3": "queijo",
    "https://www.paodeacucar.com/busca?terms=sal": "sal",
    "https://www.paodeacucar.com/busca?terms=salada &p=2": "hortifruti",
    "https://www.paodeacucar.com/busca?terms=sopa": "sopa",
    "https://www.paodeacucar.com/busca?terms=sorvete&p=3": "sorvete",
    "https://www.paodeacucar.com/categoria/alimentos/acougue/carnes/carne-aves?s=relevance&p=2": "açougue",
    "https://www.paodeacucar.com/categoria/alimentos/acougue/carnes/carne-bovina?s=relevance&p=1": "hortifruti",
    "https://www.paodeacucar.com/categoria/alimentos/hortifruti/frutas?s=relevance&p=1": "hortifruti",
    "https://www.paodeacucar.com/categoria/alimentos/hortifruti/legumes-e-verduras/hortalicas?s=relevance&p=3": "hortifruti",
    "https://www.paodeacucar.com/categoria/alimentos/hortifruti/legumes-e-verduras/legumes?s=relevance&p=1": "hortifruti",
    "https://www.paodeacucar.com/categoria/bebidas/nao-alcoolicas/refrigerantes?s=relevance&p=3": "bebidas não alcoolicas",
    "https://www.paodeacucar.com/especial/tinto-app?p=4": "bebidas alcoolicas",
    "https://www.paodeacucar.com/busca?terms=acucar_1kg" : "açucar 1kg",
    "https://www.paodeacucar.com/busca?terms=acucar_5kg" : "açucar 5kg"

}

# 1. Mapeamento: Cria uma coluna temporária ('categoria_agrupada')
#    O .map() aplica o dicionário. Se não encontrar uma chave, ele retorna NaN.
df["categoria_agrupada"] = df["categoria"].map(map_categoria)

# 2. Preenchimento e Substituição:
#    Preenche os valores NaN da coluna temporária com os valores da coluna original (df["categoria"]).
#    Isso garante que categorias não mapeadas (que resultaram em NaN) mantenham seu nome original.
#    Em seguida, a coluna 'categoria' é sobrescrita com os resultados.
df["categoria"] = df["categoria_agrupada"].fillna(df["categoria"])

# 3. Limpeza: Remove a coluna temporária
df = df.drop(columns=["categoria_agrupada"])

print("✅ Mapeamento de Categorias concluído. Categorias não mapeadas mantiveram o nome original.")


✅ Mapeamento de Categorias concluído. Categorias não mapeadas mantiveram o nome original.


In [62]:
# 1. Mapeamento
df["categoria_agrupada"] = df["categoria"].map(map_categoria)

# 2. Substituição (mantendo o nome original se não mapeado)
df["categoria"] = df["categoria_agrupada"].fillna(df["categoria"])

# 3. Limpeza
df = df.drop(columns=["categoria_agrupada"])

print("✅ Mapeamento finalizado.")

✅ Mapeamento finalizado.


In [63]:
# 1. Imprime as 20 categorias mais frequentes (para ver o que ainda está faltando)
print("\n### 📊 Contagem das 20 Categorias Mais Frequentes após Reaplicação:")
print(df["categoria"].value_counts().head(20))

# 2. Imprime as 20 categorias mais frequentes que NÃO foram mapeadas (se os valores originais aparecerem aqui)
categorias_nao_mapeadas = [cat for cat in df["categoria"].unique() if cat not in map_categoria.values()]

print("\n### 📑 Amostra de Categorias que Ficaram de Fora (NÃO mapeadas):")
# Se este bloco ainda mostrar categorias longas ou inesperadas, você precisa adicioná-las ao map_categoria.
print(categorias_nao_mapeadas[:10])


### 📊 Contagem das 20 Categorias Mais Frequentes após Reaplicação:
categoria
hortifruti                11239
bebidas não alcoolicas     9631
queijo                     4328
sal                        4157
bebidas alcoolicas         3929
massas                     3865
açougue                    2958
padaria                    2778
leite                      2721
arroz 5kg                  2713
arroz 1kg                  2663
ovos                       2554
feijão                     2401
iogurte                    2192
cafe                       1945
sorvete                    1738
cafe 500g                  1199
cha                        1160
chocolate                   911
sopa                        830
Name: count, dtype: int64

### 📑 Amostra de Categorias que Ficaram de Fora (NÃO mapeadas):
['carne bovina', 'acucar 1kg', 'acucar 5kg', 'cafe 500g', 'cerveja lata 350ml', 'agua mineral com gas', 'chocolate barra', 'cha', 'carne', 'cerveja']


In [65]:
categorias_finais = sorted(df["categoria"].unique())

print("### ✅ Lista Completa de Categorias Agrupadas:")
print(categorias_finais)

print("\n### 📊 Contagem das 20 Categorias Mais Frequentes:")
print(df["categoria"].value_counts().head(20))

### ✅ Lista Completa de Categorias Agrupadas:
['acucar 1kg', 'acucar 5kg', 'agua mineral c gas', 'agua mineral com gas', 'arroz 1kg', 'arroz 5kg', 'açougue', 'bebidas alcoolicas', 'bebidas não alcoolicas', 'cafe', 'cafe 500g', 'carne', 'carne bovina', 'cerveja', 'cerveja lata 350ml', 'cha', 'chocolate', 'chocolate 80g', 'chocolate barra', 'feijão', 'hortifruti', 'https://www.paodeacucar.com/busca?terms=acucar 1kg', 'https://www.paodeacucar.com/busca?terms=acucar 5kg', 'iogurte', 'leite', 'massas', 'oleo de soja', 'ovos', 'padaria', 'queijo', 'sal', 'sopa', 'sorvete']

### 📊 Contagem das 20 Categorias Mais Frequentes:
categoria
hortifruti                11239
bebidas não alcoolicas     9631
queijo                     4328
sal                        4157
bebidas alcoolicas         3929
massas                     3865
açougue                    2958
padaria                    2778
leite                      2721
arroz 5kg                  2713
arroz 1kg                  2663
ovos         

In [66]:
# Substitua 'dados_mercados_limpos.parquet' pelo nome que você preferir
df.to_parquet('mercados.parquet', index=False)